In [1]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageTk
from threading import Thread
from http.server import BaseHTTPRequestHandler, HTTPServer
import socketserver

In [2]:
model_path = 'kme93.keras'
model = load_model(model_path)

In [3]:
def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    pred = model.predict(np.expand_dims(img, axis=0))
    label = "Emergency" if pred > 0.5 else "Non-Emergency"
    return label

def predict_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Preprocess and predict frame
        # Display frame with prediction
        # cv2.imshow('Prediction', frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
    cap.release()
    cv2.destroyAllWindows()

def predict_webcam():
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Preprocess and predict frame
        # Display frame with prediction
        # cv2.imshow('Prediction', frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
    cap.release()
    cv2.destroyAllWindows()

In [4]:
#1

def display_video_frames(video_path=None):
    # Create a new Tkinter window for the video display
    video_window = tk.Toplevel(root)
    video_window.title("Kizito Video Prediction")

    # Create a canvas to display the video frames
    canvas = tk.Canvas(video_window, width=640, height=480)
    canvas.pack()
    
    # Create a label to display the video frames
    label_widget = tk.Label(video_window)
    label_widget.pack()

    # Function to process and display frames
    def process_frame():
        ret, frame = cap.read()
        if not ret:
            cap.release()
            video_window.destroy()
            return

        # Preprocess the frame
        frame = cv2.resize(frame, (224, 224))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = frame / 255.0

        # Predict the label
        pred = model.predict(np.expand_dims(frame, axis=0))
        label = "Emergency" if pred > 0.5 else "Non-Emergency"

        # Display the frame with the prediction
        # Note: This is a simplified example. You'll need to convert the frame to a format that Tkinter can display.
        # You might also want to overlay the prediction text on the frame.
        # canvas.create_image(0, 0, image=frame, anchor=tk.NW)

        # Schedule the next frame
        video_window.after(1000 // 30, process_frame) # 30 FPS

    # Start capturing frames
    if video_path:
        cap = cv2.VideoCapture(video_path)
    else:
        cap = cv2.VideoCapture(0) # Use webcam

    process_frame()

In [4]:
#2

def display_video_frames(video_path=None):
    # Create a new Tkinter window for the video display
    video_window = tk.Toplevel(root)
    video_window.title("Kizito Video Prediction")

    # Create a label to display the video frames
    label_widget = tk.Label(video_window)
    label_widget.pack()

    def update_frame():
        # Capture the video frame by frame
        ret, frame = cap.read()
        if not ret:
            cap.release()
            video_window.destroy()
            return

        # Preprocess the frame
        frame = cv2.resize(frame, (224, 224))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = frame / 255.0

        # Predict the label
        pred = model.predict(np.expand_dims(frame, axis=0))
        label = "Emergency" if pred > 0.5 else "Non-Emergency"

        # Convert the frame to a format that Tkinter can display
        opencv_image = cv2.cvtColor(frame, cv2.COLOR_RGB2RGBA)
        captured_image = Image.fromarray(opencv_image)
        photo_image = ImageTk.PhotoImage(image=captured_image)

        # Display the frame with the prediction
        label_widget.photo_image = photo_image
        label_widget.configure(image=photo_image)

        # Schedule the next frame
        video_window.after(1000 // 30, update_frame) # 30 FPS

    # Start capturing frames
    if video_path:
        cap = cv2.VideoCapture(video_path)
    else:
        cap = cv2.VideoCapture(0) # Use webcam

    update_frame()

In [5]:
class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        message = "<html><body><h1>Welcome to Kizito's Emergency Vehicle Detection Server</h1></body></html>"
        self.wfile.write(bytes(message, "utf8"))

In [6]:
def run_server():
    PORT = 8080
    Handler = SimpleHTTPRequestHandler
    httpd = socketserver.TCPServer(("", PORT), Handler)
    print(f"Serving on port {PORT}")
    httpd.serve_forever()

In [8]:
#Image Prediction Only

def predict_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    pred = model.predict(np.expand_dims(img, axis=0))
    label = "Emergency" if pred > 0.5 else "Non-Emergency"
    return label

def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        label = predict_image(file_path)
        result_label.config(text=f"Prediction: {label}")

def close_gui():
    root.quit()

root = tk.Tk()
root.title("Kizito Emergency Vehicle Detection")

frame = tk.Frame(root)
frame.pack()

button_image = tk.Button(frame, text="Open Image", command=open_image)
button_image.pack(side=tk.LEFT)

# Label to display the prediction result
result_label = tk.Label(root, text="")
result_label.pack(pady=20)

# Button to close the GUI
exit_button = tk.Button(root, text="Exit", command=close_gui)
exit_button.pack(pady=20)

root.mainloop()

Serving on port 8080


192.168.31.172 - - [12/Apr/2024 09:30:08] "GET / HTTP/1.1" 200 -
192.168.31.172 - - [12/Apr/2024 09:30:09] "GET /favicon.ico HTTP/1.1" 200 -
192.168.31.172 - - [12/Apr/2024 09:30:29] "GET / HTTP/1.1" 200 -
192.168.31.172 - - [12/Apr/2024 09:30:29] "GET /favicon.ico HTTP/1.1" 200 -


In [9]:
def start_server():
    server_thread = Thread(target=run_server)
    server_thread.start()

root = tk.Tk()
root.title("Kizito Emergency Vehicle Detection")

frame = tk.Frame(root)
frame.pack()

button_start_server = tk.Button(frame, text="Start Server", command=start_server)
button_start_server.pack(side=tk.LEFT)

# Existing GUI elements
button_image = tk.Button(frame, text="Open Image", command=open_image)
button_image.pack(side=tk.LEFT)

result_label = tk.Label(root, text="")
result_label.pack(pady=20)

exit_button = tk.Button(root, text="Exit", command=close_gui)
exit_button.pack(pady=20)

root.mainloop()

Exception in thread Thread-6 (run_server):
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\DELL\anaconda3\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_14348\4019646893.py", line 4, in run_server
  File "C:\Users\DELL\anaconda3\Lib\socketserver.py", line 456, in __init__
    self.server_bind()
  File "C:\Users\DELL\anaconda3\Lib\socketserver.py", line 472, in server_bind
    self.socket.bind(self.server_address)
OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted
192.168.31.172 - - [12/Apr/2024 09:31:22] "GET /favicon.ico HTTP/1.1" 200 -
192.168.31.172 - - [12/Apr/2024 09:31:24] "GET / HTTP/1.1" 200 -
192.168.31.172 - - [12/Apr/2024 09:31:24] "GET /favicon.ico HTTP/1.1" 200 -
Exception in thread Thread-7 (run_server):
Traceback (most re

In [6]:
#Image, Video and Webcam Prediction

def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        label = predict_image(file_path)
        result_label.config(text=f"Prediction: {label}")

def open_video():
    file_path = filedialog.askopenfilename()
    if file_path:
        display_video_frames(video_path=file_path)        
        
def use_webcam():
    display_video_frames()        



def close_gui():
    root.quit()

root = tk.Tk()
root.title("Kizito Emergency Vehicle Detection")

frame = tk.Frame(root)
frame.pack()

button_image = tk.Button(frame, text="Open Image", command=open_image)
button_image.pack(side=tk.LEFT)

button_video = tk.Button(frame, text="Open Video", command=open_video)
button_video.pack(side=tk.LEFT)

button_webcam = tk.Button(frame, text="Use Webcam", command=use_webcam)
button_webcam.pack(side=tk.LEFT)

# Label to display the prediction result
result_label = tk.Label(root, text="")
result_label.pack(pady=20)

# Button to close the GUI
exit_button = tk.Button(root, text="Exit", command=close_gui)
exit_button.pack(pady=20)

root.mainloop()

1/1 [==============================] - 1s 1s/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\DELL\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_6612\1751930050.py", line 12, in open_video
    display_video_frames(video_path=file_path)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_6612\4040719939.py", line 47, in display_video_frames
    update_frame()
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_6612\4040719939.py", line 30, in update_frame
    opencv_image = cv2.cvtColor(frame, cv2.COLOR_RGB2RGBA)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
cv2.error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x59191d0d::Set<3,4,-1>,struct cv::impl::